In [172]:
import numpy as np
from tqdm import tqdm
from time import sleep

In [173]:
class NN():

    #creates a 3 layer NN with 2 input neuron, n hidden neuron and 1 ouput neuron
    def __init__(self, hiddenLayer_neuron = 2, pop_size = 2):

        self.data = np.array([[0,0],
                              [0,1],
                              [1,0],
                              [1,1]])
        self.target = np.array([[0],[1],[1],[0]])

        self.hl_neuron = hiddenLayer_neuron

        self.pop_size = pop_size
        
        self.w_hidden = [np.random.uniform(size=(2,self.hl_neuron)) for _ in range(pop_size)]
        self.w_out = [np.random.uniform(size=(self.hl_neuron,1)) for _ in range(pop_size)]

        self.b_hidden = [np.random.uniform(size=(1,self.hl_neuron)) for _ in range(pop_size)]
        self.b_out = [np.random.uniform(size=(1,1)) for _ in range(pop_size)]

        self.z0s = []
        self.z1s = []

        print("Init: ")
        self.print_weights()

    #ReLU function
    def ReLU(self,x):
        return x * (x > 0) 
    
    #binary cross entropy loss function
    def bcel(self, y, y_hat):
        return -(y*np.log(y_hat) + (1-y) * np.log(1-y_hat))

    #sigmoid function
    def sigmoid(self,x):
        return 1/(1 + np.exp(-x))
    
    #Execute the NN with input x
    def execute(self,x):
        result = self.sigmoid(self.forward_pass(x)[0])
        return np.round(result)
    
    #forward pass
    def forward_pass(self,x):
        x = np.array(x)
        z0 = np.dot(x,self.w_hidden) + self.b_hidden
        hidden = self.ReLU(z0)
        z1 = np.dot(hidden,self.w_out) + self.b_out
        return z1, z0

    def neuro_train(self):
        self.mutation(self.w_hidden)
        self.mutation(self.w_out)
        self.mutation(self.b_hidden)
        self.mutation(self.b_out)
        self.print_weights()
        self.w_out.append(self.crossover(self.w_out))
        self.print_weights()


    def crossover(self, weight):
        w = np.zeros(shape=(self.pop_size,weight[0].shape[0],weight[0].shape[1]))
        lis = [i for i in range(self.pop_size)]
        np.random.shuffle(lis)
        for k in range(self.pop_size // 2):
            for i in range(weight[0].shape[0]):
                for j in range(weight[0].shape[1]):
                    if np.random.randint(0,2):
                        w[lis[2*k]][i][j] = weight[lis[2*k+1]][i][j]
                        w[lis[2*k+1]][i][j] = weight[lis[2*k]][i][j]
                    else:
                        w[lis[2*k]][i][j] = weight[lis[2*k]][i][j]
                        w[lis[2*k+1]][i][j] = weight[lis[2*k+1]][i][j]
        return w

    def mutation(self, weight):
        for k in range(len(weight)):
            for i in range(weight[k].shape[0]):
                for j in range(weight[k].shape[1]):
                    weight[k][i][j] += np.random.normal(0,0.1)

    #prints the weights of the nn  
    def print_weights(self):
        #print("W_hidden: " + str(self.w_hidden)) 
        #print("B_hidden: " + str(self.b_hidden)) 
        print("W_out: " + str(self.w_out)) 
        #print("B_out: " + str(self.b_out), end="\n\n")

    #calculates the accuracy and current loss of the nn
    def accuracy(self):
        counter = 0
    
        for i in range(len(self.data)):
            if(self.execute(self.data[i]) == self.target[i]):
                counter += 1
        loss = 0
        for i in range(len(self.data)):
            loss += self.bcel(self.target[i],self.sigmoid(self.forward_pass(self.data[i])[0]))

        return counter / len(self.data), loss / len(self.data)
    

In [174]:
nn = NN(2, pop_size= 2)

Init: 
W_out: [array([[0.27776003],
       [0.48015578]]), array([[0.77917631],
       [0.67418224]])]


In [175]:
nn.neuro_train()
#nn.train(epochs = -1)

W_out: [array([[0.20541885],
       [0.46573567]]), array([[0.72962354],
       [0.63777481]])]
W_out: [array([[0.20541885],
       [0.46573567]]), array([[0.72962354],
       [0.63777481]]), array([[[0.20541885],
        [0.63777481]],

       [[0.72962354],
        [0.46573567]]])]


In [176]:
print(nn.execute([0,0]))
print(nn.execute([0,1]))
print(nn.execute([1,0]))
print(nn.execute([1,1]))

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (3, 2) + inhomogeneous part.

In [ ]:
a = [np.random.uniform(size=(2,8)) for _ in range(20)]
print(len(a))

20
